In [ ]:
import pandas as pd
import numpy as np
from functions import *

In [ ]:
loan_df = pd.read_csv('datasets/loan.csv', sep=',')  # 8 categorical attributes, 4 numerical attributes, and a Loan ID attribute Loan_Status - target
loan_df.head() 

loan_df.rename(columns={'Loan_Status': 'target'}, inplace=True)
column = loan_df.pop('target')
loan_df.insert(len(loan_df.columns), 'target', column)
loan_df.drop(['Loan_ID'], axis=1, inplace=True)

In [ ]:
type_of_col = {
    'Gender': 0,
    'Married': 0,
    'Dependents': 0,
    'Education': 0,
    'Self_Employed': 0,
    'ApplicantIncome': 1,
    'CoapplicantIncome': 1, 
    'LoanAmount': 1,
    'Loan_Amount_Term': 1,
    'Credit_History': 0,
    'Property_Area': 0, 
    'target': 0
}

In [ ]:
to_normalize = []
to_encode = []
for col in loan_df.columns:
    if(type_of_col[col]):
        to_normalize.append(col)
    else:
        to_encode.append(col)

In [ ]:
norm_loan_df = normalise(loan_df, to_normalize)
encoded_loan_df = encode_one_hot(norm_loan_df, to_encode)
encoded_col_names = encoded_loan_df.columns

In [ ]:
smaller_dfs = distribute_records(norm_loan_df, 10)

In [ ]:
norm_loan_df['target'].unique()

In [ ]:
def arch_arr(input, output):
    architectures = [
        [input, 13, output],
        [input, 13, output],
        [input, 8, 9, output],
        [input, 8, 9, 5, output],
        [input, 8, 9, 5, output],
        [input, 12, 18, 13, 15, output],
    ]
    return architectures


architectures = arch_arr(22, 2)
learning_rate = 0.1
regularization_vals = [0, 0.1, 0,0, 0.1,0]

idx = 0

f1_score_matrix = np.zeros((10, len(architectures)))
accuracy_matrix = np.zeros((10, len(architectures)))

In [ ]:

for architecture in architectures:      
    regularization_param = regularization_vals[idx]
    test_accuracy_across_folds = []
    for fold in range(0,10):
        weights = initialize_weights_minustoplus(architecture)
        gradients = [np.zeros_like(theta) for theta in weights] 
        test_df = smaller_dfs[fold]
        train_df = pd.concat(smaller_dfs[:fold] + smaller_dfs[fold+1:])
        test_df.reset_index(inplace=True, drop=True)
        train_df.reset_index(inplace=True, drop=True)
        
        test_df = encode_one_hot(test_df, to_encode)
        test_df = correct_encoding(test_df, encoded_col_names)
        
        train_df = encode_one_hot(train_df,to_encode)
        train_df = correct_encoding(train_df, encoded_col_names)

        x_cols_tr = train_df.columns[:architecture[0]]
        y_cols_tr = train_df.columns[-architecture[-1]:]

        x_train = train_df[x_cols_tr].values
        y_train = train_df[y_cols_tr].values

        x_cols_ts = test_df.columns[:architecture[0]]
        y_cols_ts = test_df.columns[-architecture[-1]:]

        x_test = test_df[x_cols_ts].values
        y_test = test_df[y_cols_ts].values

        cost_before = calculate_cost(x_train, y_train, weights, regularization_param)

        for i in range(500):
            weights = backpropagation(x_train, y_train, architecture, weights, learning_rate, regularization_param)
            cost_itr = calculate_cost(x_train, y_train, weights, regularization_param)

        cost_after_forward_pass = calculate_cost(x_train, y_train, weights, regularization_param)

        tr_results = []
        for i in range(len(x_train)):
            pred_i = forward_pass(x_train[i], weights)
            max_index_pred = np.argmax(pred_i)
            max_index_true = np.argmax(y_train[i])
            tr_results.append((max_index_pred+1, max_index_true+1))
        train_results = pd.DataFrame(tr_results, columns=['predicted_target', 'target'])
        train_accuracy, train_f1 = calculate_performance(train_results)
        
        ts_results = []
        for i in range(len(x_test)):
            pred_i = forward_pass(x_test[i], weights)
            max_index_pred = np.argmax(pred_i)
            max_index_true = np.argmax(y_test[i])
            ts_results.append((max_index_pred+1, max_index_true+1))
        test_results = pd.DataFrame(ts_results, columns=['predicted_target', 'target'])
        test_accuracy, test_f1 = calculate_performance(test_results)
        test_accuracy_across_folds.append(test_accuracy)
        accuracy_matrix[fold, idx]  = test_accuracy
        f1_score_matrix[fold, idx] = test_f1
    idx += 1

In [ ]:
average_accuracy = np.mean(accuracy_matrix, axis=0)
average_f1 = np.mean(f1_score_matrix, axis=0)

In [ ]:
for i, arch in enumerate(architectures):
    description = f"Architecture {i+1}: "
    description += f"Layers: {arch[:]}, Regularization Param: {regularization_vals[i]}"
    print(description)

In [ ]:
df = pd.DataFrame(accuracy_matrix, columns=[f'Architecture {i+1}' for i in range(len(architectures))])

# Calculate mean for each architecture and add as last row
mean_row = df.mean(axis=0)
mean_row.name = 'Mean'
df = pd.concat([df, mean_row.to_frame().T])

# Display DataFrame
df

In [ ]:
df = pd.DataFrame(f1_score_matrix, columns=[f'Architecture {i+1}' for i in range(len(architectures))])

# Calculate mean for each architecture and add as last row
mean_row = df.mean(axis=0)
mean_row.name = 'Mean'
df = pd.concat([df, mean_row.to_frame().T])

# Display DataFrame
df

### learnign curve

In [ ]:
norm_loan_df = normalise(loan_df, to_normalize)
encoded_loan_df = encode_one_hot(norm_loan_df, to_encode)

In [ ]:
learning_rate = 0.1
regularization_param = 0
architecture = [22, 13, 2]

weights = initialize_weights_minustoplus(architecture)



x_cols = encoded_loan_df.columns[:architecture[0]]
y_cols = encoded_loan_df.columns[-architecture[-1]:]

x = encoded_loan_df[x_cols].values
y = encoded_loan_df[y_cols].values

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)


cost_j_on_test_set = []
iterations_val = []


# Initialize weights and gradients
original_weights = initialize_weights_minustoplus(architecture)
original_gradients = [np.zeros_like(theta) for theta in weights]

# Loop over the range of indices
for offset in range(0,901, 120):
    # Set weights and gradients to original values
    weights = original_weights
    gradients = original_gradients
    
    # Iterate over the data in batches of size 20
    for idx in range(10, len(x_train), 20):
        # Perform backpropagation on a subset of the training data
        weights = backpropagation(x_train[:idx], y_train[:idx], architecture, weights, learning_rate, regularization_param)
    
    # Calculate cost on the test set
    cost_itr = calculate_cost(x_test, y_test, weights, regularization_param)
    
    # Append cost and iteration values to lists
    cost_j_on_test_set.append(cost_itr)
    iterations_val.append(offset + idx)

In [ ]:
plt.figure(figsize=(8, 6))
plt.plot(iterations_val, cost_j_on_test_set, marker='o', linestyle='-')
plt.title('Learning Curve')
plt.xlabel('Number of Training Examples')
plt.ylabel('Cost Function J value')
plt.grid(True)
plt.show()